In [1]:
!pip install tsplib95

Defaulting to user installation because normal site-packages is not writeable


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [15]:
import tsplib95
import numpy as np

def get_coordinates(problem):
    """Extract coordinates from the TSP problem."""

    return {node: problem.node_coords[node] for node in problem.get_nodes()}

def euclidean_distance(start_coords, end_coords):
    """Compute the Euclidean distance between two points."""
    return np.sqrt((start_coords[0] - end_coords[0]) ** 2 + (start_coords[1] - end_coords[1]) ** 2)

def compute_distance_matrix(problem):
    """Compute the full distance matrix for the problem."""
 
    coordinates = get_coordinates(problem)

    n = len(coordinates)
    distance_matrix = np.zeros((n, n))

    for i in range(n):
        for j in range(i + 1, n):
            start = list(coordinates.keys())[i]
            end = list(coordinates.keys())[j]
            dist = euclidean_distance(coordinates[start], coordinates[end])
            distance_matrix[i, j] = dist
            distance_matrix[j, i] = dist  # Since the distance is symmetric
    
    return distance_matrix


problem = tsplib95.load('./rd100.tsp')


distance_matrix = compute_distance_matrix(problem)


print(distance_matrix)


[[   0.         1134.34650195  421.26654102 ...  766.44389621
   543.59588049  820.04715234]
 [1134.34650195    0.          807.31092407 ...  414.90794398
   643.11343403  414.45806164]
 [ 421.26654102  807.31092407    0.         ...  397.46091036
   164.23692755  621.7397887 ]
 ...
 [ 766.44389621  414.90794398  397.46091036 ...    0.
   235.7065088   386.12785427]
 [ 543.59588049  643.11343403  164.23692755 ...  235.7065088
     0.          485.04202491]
 [ 820.04715234  414.45806164  621.7397887  ...  386.12785427
   485.04202491    0.        ]]


In [21]:
def get_distance_matrix(dataset):
    problem = tsplib95.load(dataset)
    distance_matrix = compute_distance_matrix(problem)
    print("the distance matrix is \n")
    print(distance_matrix)
    return distance_matrix

### Basic Implementation

In [12]:
import random
import numpy as np

# Function to calculate the total distance of a path (sum of distances between consecutive cities)
def total_distance(path, distance_matrix):
    return sum(distance_matrix[path[i], path[i + 1]] for i in range(len(path) - 1)) + distance_matrix[path[-1], path[0]]

# Function to get neighbors (all possible neighboring solutions)
def get_neighbors(path):
    neighbors = []
    for i in range(len(path)):
        for j in range(i + 1, len(path)):
            # Swap two cities to create a neighbor
            new_path = path[:]
            new_path[i], new_path[j] = new_path[j], new_path[i]
            neighbors.append(new_path)
    return neighbors

# Simple Hill Climbing
def simple_hill_climbing(initial_path, distance_matrix):
    current_path = initial_path
    current_distance = total_distance(current_path, distance_matrix)
    
    while True:
        neighbors = get_neighbors(current_path)
        next_path = None
        next_distance = current_distance
        
        for neighbor in neighbors:
            neighbor_distance = total_distance(neighbor, distance_matrix)
            if neighbor_distance < next_distance:
                next_path = neighbor
                next_distance = neighbor_distance
                break # Stop after finding the first improvement

        
        if next_path is None:  # No improvement, so stop
            break
        
        current_path = next_path
        current_distance = next_distance
    
    return current_path, current_distance

# Stochastic Hill Climbing
def stochastic_hill_climbing(initial_path, distance_matrix):
    current_path = initial_path
    current_distance = total_distance(current_path, distance_matrix)
    
    while True:
        neighbors = get_neighbors(current_path)
        next_path = random.choice(neighbors)  # Randomly choose a neighbor
        next_distance = total_distance(next_path, distance_matrix)
        
        if next_distance < current_distance:
            current_path = next_path
            current_distance = next_distance
        
        # Stopping condition (you can adjust this)
        if random.random() > 0.95:  # Let's stop after a few random moves
            break
    
    return current_path, current_distance

# Steepest Ascent Hill Climbing
def steepest_ascent_hill_climbing(initial_path, distance_matrix):
    current_path = initial_path
    current_distance = total_distance(current_path, distance_matrix)
    
    while True:
        neighbors = get_neighbors(current_path)
        best_path = None
        best_distance = current_distance
        
        for neighbor in neighbors:
            neighbor_distance = total_distance(neighbor, distance_matrix)
            if neighbor_distance < best_distance:
                best_path = neighbor
                best_distance = neighbor_distance
        
        if best_path is None:  # No better neighbor found
            break
        
        current_path = best_path
        current_distance = best_distance
    
    return current_path, current_distance

# Example usage
def run_hill_climbing(distance_matrix):
    # Generate a random initial solution (random permutation of cities)
    initial_path = list(range(len(distance_matrix)))
    random.shuffle(initial_path)

    # Simple Hill Climbing
    best_path, best_distance = simple_hill_climbing(initial_path, distance_matrix)
    print(f"Simple Hill Climbing: Best path: {best_path}, Distance: {best_distance}")
    
    # Stochastic Hill Climbing
    best_path, best_distance = stochastic_hill_climbing(initial_path, distance_matrix)
    print(f"Stochastic Hill Climbing: Best path: {best_path}, Distance: {best_distance}")
    
    # Steepest Ascent Hill Climbing
    best_path, best_distance = steepest_ascent_hill_climbing(initial_path, distance_matrix)
    print(f"Steepest Ascent Hill Climbing: Best path: {best_path}, Distance: {best_distance}")


run_hill_climbing(distance_matrix)


Simple Hill Climbing: Best path: [59, 14, 62, 96, 85, 81, 84, 11, 13, 86, 61, 17, 0, 70, 67, 82, 64, 79, 80, 90, 46, 27, 36, 18, 45, 49, 55, 26, 32, 12, 66, 20, 71, 69, 37, 53, 72, 51, 10, 60, 4, 77, 73, 19, 9, 78, 5, 97, 93, 52, 98, 48, 74, 3, 31, 8, 25, 16, 91, 2, 89, 50, 47, 29, 21, 40, 33, 6, 41, 23, 24, 42, 39, 54, 95, 38, 99, 44, 15, 30, 83, 43, 28, 34, 22, 1, 88, 57, 87, 65, 75, 92, 76, 58, 94, 56, 35, 63, 68, 7], Distance: 12712.614569952177
Stochastic Hill Climbing: Best path: [19, 48, 80, 67, 51, 7, 71, 91, 78, 53, 47, 85, 17, 84, 73, 5, 43, 2, 44, 83, 50, 57, 34, 18, 61, 54, 20, 87, 39, 79, 59, 3, 68, 9, 89, 56, 98, 77, 75, 41, 21, 32, 88, 8, 16, 94, 10, 93, 58, 1, 70, 46, 22, 37, 86, 42, 45, 12, 38, 92, 63, 90, 26, 72, 96, 35, 6, 23, 76, 13, 27, 52, 82, 29, 66, 24, 62, 69, 25, 36, 28, 95, 31, 74, 99, 11, 4, 15, 65, 81, 40, 49, 55, 33, 60, 0, 97, 30, 64, 14], Distance: 54160.52551871367
Steepest Ascent Hill Climbing: Best path: [61, 17, 0, 59, 7, 86, 8, 16, 37, 53, 72, 19, 7

## Experimental Evaluation

In [20]:
import random
import numpy as np
import time

# Function to calculate the total distance of a path (sum of distances between consecutive cities)
def total_distance(path, distance_matrix):
    return sum(distance_matrix[path[i], path[i + 1]] for i in range(len(path) - 1)) + distance_matrix[path[-1], path[0]]

# Function to get neighbors (all possible neighboring solutions)
def get_neighbors(path):
    neighbors = []
    for i in range(len(path)):
        for j in range(i + 1, len(path)):
            # Swap two cities to create a neighbor
            new_path = path[:]
            new_path[i], new_path[j] = new_path[j], new_path[i]
            neighbors.append(new_path)
    return neighbors

# Simple Hill Climbing
def simple_hill_climbing(initial_path, distance_matrix):
    current_path = initial_path
    current_distance = total_distance(current_path, distance_matrix)
    iterations = 0

    while True:
        iterations += 1
        neighbors = get_neighbors(current_path)
        next_path = None
        next_distance = current_distance
        
        for neighbor in neighbors:
            neighbor_distance = total_distance(neighbor, distance_matrix)
            if neighbor_distance < next_distance:
                next_path = neighbor
                next_distance = neighbor_distance
                break
        
        if next_path is None:  # No improvement, so stop
            break
        
        current_path = next_path
        current_distance = next_distance
    
    return current_path, current_distance, iterations

# Steepest Ascent Hill Climbing
def steepest_ascent_hill_climbing(initial_path, distance_matrix):
    current_path = initial_path
    current_distance = total_distance(current_path, distance_matrix)
    iterations = 0

    while True:
        iterations += 1
        neighbors = get_neighbors(current_path)
        best_path = None
        best_distance = current_distance
        
        # Evaluate all neighbors and select the best one
        for neighbor in neighbors:
            neighbor_distance = total_distance(neighbor, distance_matrix)
            if neighbor_distance < best_distance:
                best_path = neighbor
                best_distance = neighbor_distance
        
        # If no better path is found, exit the loop
        if best_path is None:  
            break
        
        # If the best path is the same as the current path, stop
        if best_path == current_path:  
            break
        
        current_path = best_path
        current_distance = best_distance
    
    return current_path, current_distance, iterations

# Stochastic Hill Climbing
def stochastic_hill_climbing(initial_path, distance_matrix):
    current_path = initial_path
    current_distance = total_distance(current_path, distance_matrix)
    iterations = 0

    while True:
        iterations += 1
        neighbors = get_neighbors(current_path)
        next_path = random.choice(neighbors)  # Randomly choose a neighbor
        next_distance = total_distance(next_path, distance_matrix)
        
        if next_distance < current_distance:
            current_path = next_path
            current_distance = next_distance
        
        # Stopping condition (you can adjust this)
        if random.random() > 0.95:  # Let's stop after a few random moves
            break
    
    return current_path, current_distance, iterations

# Example usage
def run_hill_climbing(distance_matrix):
    # Generate a random initial solution (random permutation of cities)
    initial_path = list(range(len(distance_matrix)))
    random.shuffle(initial_path)

    # Ensure initial_path is valid
    print(f"Initial path: {initial_path}")  # Debugging line

    # Measure the time for each algorithm
    start_time = time.time()
    
    # Simple Hill Climbing
    best_path, best_distance, iterations = simple_hill_climbing(initial_path, distance_matrix)
    simple_time = time.time() - start_time
    print(f"Simple Hill Climbing: Best path: {best_path}, Distance: {best_distance}, Iterations: {iterations}, Time: {simple_time:.5f} seconds")
    
    start_time = time.time()
    
    # Stochastic Hill Climbing
    best_path, best_distance, iterations = stochastic_hill_climbing(initial_path, distance_matrix)
    stochastic_time = time.time() - start_time
    print(f"Stochastic Hill Climbing: Best path: {best_path}, Distance: {best_distance}, Iterations: {iterations}, Time: {stochastic_time:.5f} seconds")
    
    start_time = time.time()
    
    # Steepest Ascent Hill Climbing
    best_path, best_distance, iterations = steepest_ascent_hill_climbing(initial_path, distance_matrix)
    steepest_time = time.time() - start_time
    print(f"Steepest Ascent Hill Climbing: Best path: {best_path}, Distance: {best_distance}, Iterations: {iterations}, Time: {steepest_time:.5f} seconds")


# Run the hill climbing algorithms
run_hill_climbing(distance_matrix)


Initial path: [15, 37, 70, 67, 17, 0, 24, 74, 2, 42, 66, 46, 91, 32, 54, 6, 97, 72, 25, 86, 11, 55, 20, 58, 56, 36, 48, 71, 79, 51, 47, 73, 26, 16, 8, 5, 19, 43, 83, 84, 40, 41, 94, 10, 1, 93, 65, 49, 44, 30, 69, 81, 57, 12, 29, 21, 63, 35, 76, 28, 59, 13, 22, 82, 52, 45, 92, 50, 7, 99, 80, 75, 18, 89, 62, 53, 60, 61, 95, 68, 14, 33, 31, 38, 23, 77, 39, 88, 98, 85, 3, 90, 64, 87, 9, 34, 78, 27, 4, 96]
Simple Hill Climbing: Best path: [6, 41, 23, 24, 42, 39, 54, 95, 38, 4, 80, 79, 64, 50, 82, 0, 86, 13, 11, 84, 81, 20, 85, 62, 14, 61, 17, 70, 67, 47, 29, 31, 8, 25, 73, 32, 78, 5, 94, 65, 87, 22, 1, 88, 57, 75, 58, 76, 27, 36, 18, 55, 26, 9, 63, 35, 56, 46, 52, 98, 2, 3, 77, 19, 91, 16, 71, 69, 37, 53, 72, 49, 45, 92, 43, 34, 28, 99, 89, 68, 7, 59, 96, 66, 12, 15, 97, 93, 74, 48, 90, 51, 44, 83, 30, 10, 60, 21, 40, 33], Distance: 13512.92081663388, Iterations: 506, Time: 7.43075 seconds
Stochastic Hill Climbing: Best path: [15, 37, 70, 67, 17, 61, 24, 74, 2, 42, 66, 46, 91, 32, 54, 6, 97

## Experimental setup for testing across different datasets

In [23]:
import random
import time
import pandas as pd

datasets = ['./rd100.tsp','./eil101.tsp','./a280.tsp','./d198.tsp','./ch150.tsp']
table = []

def run_experiment(table, datasets):
    for dataset in datasets:
        print(f"Dataset is: {dataset} \n")
        distance_matrix = get_distance_matrix(dataset)
        
        # Generate a random initial solution (random permutation of cities)
        initial_path = list(range(len(distance_matrix)))
        random.shuffle(initial_path)

        # Ensure initial_path is valid
        print(f"Initial path: {initial_path}")  # Debugging line

        # Measure the time for each algorithm
        start_time = time.time()

        # Simple Hill Climbing
        best_path, best_distance, iterations = simple_hill_climbing(initial_path, distance_matrix)
        simple_time = time.time() - start_time
        print(f"Simple Hill Climbing: Best path: {best_path}, Distance: {best_distance}, Iterations: {iterations}, Time: {simple_time:.5f} seconds")
        
        table.append([dataset, 'Simple Hill Climbing', best_path, best_distance, iterations, simple_time])

        start_time = time.time()

        # Stochastic Hill Climbing
        best_path, best_distance, iterations = stochastic_hill_climbing(initial_path, distance_matrix)
        stochastic_time = time.time() - start_time
        print(f"Stochastic Hill Climbing: Best path: {best_path}, Distance: {best_distance}, Iterations: {iterations}, Time: {stochastic_time:.5f} seconds")
        table.append([dataset, 'Stochastic Hill Climbing', best_path, best_distance, iterations, stochastic_time])

        start_time = time.time()

        # Steepest Ascent Hill Climbing
        best_path, best_distance, iterations = steepest_ascent_hill_climbing(initial_path, distance_matrix)
        steepest_time = time.time() - start_time
        print(f"Steepest Ascent Hill Climbing: Best path: {best_path}, Distance: {best_distance}, Iterations: {iterations}, Time: {steepest_time:.5f} seconds")
        table.append([dataset, 'Steepest Ascent Hill Climbing', best_path, best_distance, iterations, steepest_time])
        
    # Convert table to DataFrame and print it
    df = pd.DataFrame(table, columns=['Dataset', 'Algorithm', 'Best Path', 'Best Distance', 'Iterations', 'Time (seconds)'])
    print(df)

# Example call to run the experiment
run_experiment(table, datasets)


Dataset is: ./rd100.tsp 

the distance matrix is 

[[   0.         1134.34650195  421.26654102 ...  766.44389621
   543.59588049  820.04715234]
 [1134.34650195    0.          807.31092407 ...  414.90794398
   643.11343403  414.45806164]
 [ 421.26654102  807.31092407    0.         ...  397.46091036
   164.23692755  621.7397887 ]
 ...
 [ 766.44389621  414.90794398  397.46091036 ...    0.
   235.7065088   386.12785427]
 [ 543.59588049  643.11343403  164.23692755 ...  235.7065088
     0.          485.04202491]
 [ 820.04715234  414.45806164  621.7397887  ...  386.12785427
   485.04202491    0.        ]]
Initial path: [98, 51, 96, 16, 86, 43, 70, 25, 88, 35, 64, 93, 8, 44, 26, 15, 53, 50, 95, 0, 82, 55, 10, 61, 90, 89, 65, 69, 19, 7, 77, 73, 47, 85, 21, 79, 68, 56, 41, 45, 76, 18, 46, 97, 1, 49, 92, 17, 14, 29, 67, 75, 62, 63, 78, 83, 24, 33, 87, 57, 38, 81, 31, 54, 39, 5, 6, 34, 32, 20, 3, 2, 94, 4, 58, 72, 36, 99, 37, 22, 91, 60, 12, 9, 71, 13, 80, 28, 30, 59, 27, 23, 52, 11, 84, 74, 48, 6

KeyboardInterrupt: 